<a href="https://colab.research.google.com/github/ramnarayan-code/llm_reflection/blob/main/langgraph_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langchain langgraph langchain_openai langchain_experimental langsmith pandas

In [3]:
import os
os.environ['OPENAI_API_KEY'] = ''
# https://medium.com/@anuragmishra_27746/future-of-coding-multi-agent-llm-framework-using-langgraph-092da9493663

In [6]:
import os
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_agent_executor
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
import operator


In [7]:
llm=ChatOpenAI()

In [8]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

class ProductOwner(BaseModel):

    user_stories: str = Field(
        description="Detailed user stories"
    )
product_owner_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: You are a product owner. You need to create user stories
**Task**: As a product owner, you are required to create 5 user stories based on the use case

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the use case.
2. **Define user story and assign priority**: define user story in the below format following the agile standards and assign the priority level out of the values: high, medium, low.
*FORMAT*
User story:
Priority:

*USE CASE*
{use_case}

'''
)
product_owner_agent = create_structured_output_runnable(
    ProductOwner, llm, product_owner_prompt
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. You can read more about the method here: https://python.langchain.com/docs/modules/model_io/chat/structured_output/ Please follow our extraction use case documentation for more guidelines on how to do information extraction with LLMs. https://python.langchain.com/docs/use_cases/extraction/. If you notice other issues, please provide feedback here: https://github.com/langchain-ai/langchain/discussions/18154
  warn_deprecated(


In [9]:
user_stories = product_owner_agent.invoke({'use_case':'Design a Video streaming platform'})


In [10]:
print(user_stories.user_stories)

User story 1: As a user, I want to be able to create a personalized profile on the video streaming platform.
Priority: Medium

User story 2: As a user, I want to search for and discover new videos based on my interests and preferences.
Priority: High

User story 3: As a user, I want to be able to create and manage playlists of my favorite videos.
Priority: Medium

User story 4: As a user, I want to receive personalized recommendations for videos based on my viewing history.
Priority: High

User story 5: As a user, I want to be able to like, comment, and share videos with other users on the platform.
Priority: High


In [27]:
class Developer(BaseModel):
    """Plan to follow in future"""

    technical_solution: str = Field(
        description="Detailed optmized error-free Python code on the provided requirements"
    )


from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

developer_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: You are a software architect. You need to assess the user stories
**Task**: As an architect, you need to assess the user stories and provide the technical feasibility report

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the use case and its related user stories.
2. **Propose technical solution**: propose a detailed technical solution for the use case and also define the potential technology stack
in the below output format.
*FORMAT*
High level Technical Solution: <Proposal>
Tech Stack: <tech-stack>
3. **Define estimates**: estimate the efforts to complete the implementation of the proposed technical solution in the below output format:
*FORMAT*
ETA: <eta>

*USE CASE*
{use_case}

*USE STORIES*
{user_stories}
'''
)
developer_agent = create_structured_output_runnable(
    Developer, llm, developer_prompt
)


In [28]:
developer_report = developer_agent.invoke({'user_stories':user_stories.user_stories, 'use_case':'Design a Video streaming platform'})


In [30]:
print(developer_report.technical_solution)

In [31]:
class AgentState(TypedDict):
    use_case: str
    user_stories: str
    technical_solution: str

In [35]:
def product_owner(state):
    print(f'Entering PO')
    use_case = state['use_case']
    print(use_case)
    po_outcome = product_owner_agent.invoke({'use_case':use_case})
    return {'user_stories':po_outcome.user_stories}

def developer(state):
    print(f'Entering Dev')
    user_stories = state['user_stories']
    use_case = state['use_case']
    developer_outcome = developer_agent.invoke({'user_stories':user_stories, 'use_case':use_case})
    return {'technical_solution':developer_outcome.technical_solution}

In [36]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("product_owner", product_owner)
workflow.add_node("developer", developer)

# Build graph
workflow.set_entry_point("product_owner")
workflow.add_edge("product_owner", "developer")
workflow.add_edge("developer", END)

app = workflow.compile()


In [38]:
from langchain_core.messages import HumanMessage

for s in app.stream(
    {"use_case": [HumanMessage(content="Design a CRM system")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

Entering PO
[HumanMessage(content='Design a CRM system')]
{'product_owner': {'user_stories': "User story: As a sales manager, I want to be able to track customer interactions and communications in the CRM system so that I can have a complete view of customer history.\nPriority: High\n\nUser story: As a customer service representative, I want to access customer information quickly in the CRM system so that I can provide personalized support.\nPriority: High\n\nUser story: As a marketing strategist, I want to analyze customer data and behavior patterns in the CRM system to create targeted marketing campaigns.\nPriority: Medium\n\nUser story: As a system administrator, I want to be able to customize the CRM system settings to match our organization's specific requirements.\nPriority: Medium\n\nUser story: As a CEO, I want to view real-time reports and analytics in the CRM system to make data-driven business decisions.\nPriority: High"}}
----
Entering Dev
{'developer': {'technical_solution